# Week 12: Capstone Project Part 5.2

- Done by: A Alkaff Ahamed
- Grade: Pending
- 2 July 2025


## Learning Outcome Addressed
- Apply techniques for effective communication and coordination among multiple agents
- Leverage LLMs for task planning, execution and autonomous problem-solving
- Assess the limitations, challenges and emerging trends in multi-agent AI systems


## Objective 

Build a basic function-calling agent that retrieves and processes weather information, demonstrating the practical implementation of agent capabilities. 

**Prerequisites** 

- OpenAI API key 
- Python 3.8+ 
- SQLite3 (included with Python) 
- Basic SQL knowledge 

**Setup** 

1. Create `config.py`: 

```
OPENAI_API_KEY = "your-key-here" 
```

1. Set up your virtual environment and install requirements:

```
# Activate virtual environment 
source bin/activate   # On Unix/Mac 
# OR 
.\bin\activate       # On Windows 
 
# Install requirements 
pip install openai==0.28 
```

1. Keep your terminal with the activated virtual environment open - you’ll need it to run the test commands throughout this exercise. 

## Exercise Structure

Part 1: Setting Up the Database 

Create `database.py` and add: 

```python
import sqlite3 
 
def setup_database(): 
    conn = sqlite3.connect('company.db') 
    c = conn.cursor() 
     
    # Create sample tables 
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS employees ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            department TEXT, 
            salary REAL 
        ) 
    ''') 
     
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS departments ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            budget REAL 
        ) 
    ''') 
     
    # Insert sample data 
    c.execute("INSERT OR IGNORE INTO employees VALUES (1, 'John Doe', 'Engineering', 75000)") 
    c.execute("INSERT OR IGNORE INTO employees VALUES (2, 'Jane Smith', 'Marketing', 65000)") 
    c.execute("INSERT OR IGNORE INTO departments VALUES (1, 'Engineering', 1000000)") 
    c.execute("INSERT OR IGNORE INTO departments VALUES (2, 'Marketing', 500000)") 
     
    conn.commit() 
    conn.close() 
```

Test Part 1: Add this code at the bottom of database.py and run it: 

```python
# Test database setup 
if __name__ == "__main__": 
    setup_database() 
     
    # Verify the setup 
    conn = sqlite3.connect('company.db') 
    c = conn.cursor() 
     
    print("Employees table:") 
    c.execute("SELECT * FROM employees") 
    print(c.fetchall()) 
     
    print("\nDepartments table:") 
    c.execute("SELECT * FROM departments") 
    print(c.fetchall()) 
     
    conn.close() 
```

Part 2: Creating the SQL Generator 

Create `sql_agent.py` and add: 

```python
import openai 
from config import OPENAI_API_KEY 
 
openai.api_key = OPENAI_API_KEY 
 
def get_schema(): 
    return """ 
    Table: employees 
    Columns: 
    - id (INTEGER PRIMARY KEY) 
    - name (TEXT) 
    - department (TEXT) 
    - salary (REAL) 
 
    Table: departments 
    Columns: 
    - id (INTEGER PRIMARY KEY) 
    - name (TEXT) 
    - budget (REAL) 
    """ 
 
def generate_sql(question): 
    response = openai.ChatCompletion.create( 
        model="gpt-4", 
        messages=[ 
            {"role": "system", "content": f"""You are a SQL expert. Use this schema:\n{get_schema()} 
            Return ONLY the SQL query without any explanation or markdown formatting."""}, 
            {"role": "user", "content": f"Generate SQL for: {question}"} 
        ] 
    ) 
    sql = response.choices[0].message.content.strip() 
     
    # Remove any markdown code block syntax 
    sql = sql.replace('```sql', '').replace('```SQL', '').replace('```', '') 
     
    # Remove any explanatory text before or after the SQL 
    sql_lines = [line.strip() for line in sql.split('\n') if line.strip()] 
    sql = ' '.join(sql_lines) 
     
    return sql 
```

Test Part 2: Add this code and run it: 

```python
# Test SQL generation 
if __name__ == "__main__": 
    test_questions = [ 
        "List all employees", 
        "Show departments with budgets over 750000" 
    ] 
     
    for question in test_questions: 
        print(f"\nQuestion: {question}") 
        print(f"Generated SQL: {generate_sql(question)}") 
```

Part 3: Implementing Query Execution 

Add to `sql_agent.py`: 

```python
import sqlite3 
 
def validate_sql(sql): 
    # Basic safety checks 
    sql_lower = sql.lower() 
    if any(word in sql_lower for word in ['drop', 'delete', 'update', 'insert']): 
        raise ValueError("Only SELECT queries are allowed") 
    return sql 
 
def execute_query(sql): 
    sql = validate_sql(sql) 
    conn = sqlite3.connect('company.db') 
    try: 
        cursor = conn.cursor() 
        cursor.execute(sql) 
        results = cursor.fetchall() 
        return results 
    except Exception as e: 
        return f"Error: {str(e)}" 
    finally: 
        conn.close() 
 
def format_results(results): 
    if not isinstance(results, list): 
        return str(results) 
    if not results: 
        return "No results found" 
     
    # If it's a single column result 
    if len(results[0]) == 1: 
        return "\n".join([str(row[0]) for row in results]) 
     
    # For multiple columns, try to format as a table 
    # Get column names from the first result 
    if isinstance(results[0], tuple): 
        # Format each row with proper spacing 
        formatted_rows = [] 
        for row in results: 
            row_items = [] 
            for item in row: 
                if isinstance(item, float): 
                    row_items.append(f"${item:,.2f}" if "salary" in str(row) or "budget" in str(row) else f"{item:.2f}") 
                else: 
                    row_items.append(str(item)) 
            formatted_rows.append("\t".join(row_items)) 
        return "\n".join(formatted_rows) 
     
    return "\n".join([str(row) for row in results]) 
```

Test Part 3: Replace the previous test code with: 

```python
# Test query execution 
if __name__ == "__main__": 
    test_sql = [ 
        "SELECT * FROM employees WHERE salary > 70000", 
        "SELECT d.name, COUNT(e.id) FROM departments d LEFT JOIN employees e ON d.name = e.department GROUP BY d.name" 
    ] 
     
    for sql in test_sql: 
        print(f"\nExecuting SQL: {sql}") 
        print(f"Results: {execute_query(sql)}") 
```

Part 4: Creating the Complete Agent 

Add the final agent code: 

```python
def query_agent(question): 
    try: 
        # Generate SQL 
        sql = generate_sql(question) 
        print(f"Generated SQL: {sql}\n") 
         
        # Execute and format results 
        results = execute_query(sql) 
        return format_results(results) 
    except Exception as e: 
        return f"Error: {str(e)}" 
 
# Test the complete agent 
if __name__ == "__main__": 
    test_questions = [ 
        "What is the average salary in each department?", 
        "Which department has the highest budget?", 
        "List all employees earning more than 70000", 
        "DROP TABLE employees"  # This should be caught by validation 
    ] 
     
    for question in test_questions: 
        print(f"\nQuestion: {question}") 
        print(f"Answer: {query_agent(question)}") 
```

Example output: 

```
Question: What is the average salary in each department? 
Generated SQL: SELECT department, AVG(salary) FROM employees GROUP BY department; 
 
Answer:  
Engineering $75,000.00 
Marketing   $65,000.00 
 
Question: Which department has the highest budget? 
Generated SQL: SELECT name FROM departments ORDER BY budget DESC LIMIT 1; 
 
Answer: Engineering 
 
Question: List all employees earning more than 70000 
Generated SQL: SELECT * FROM employees WHERE salary > 70000; 
 
Answer: 1   John Doe    Engineering $75,000.00 
 
Question: DROP TABLE employees 
Generated SQL: DROP TABLE employees; 
 
Answer: Error: Only SELECT queries are allowed 
```


1. **Troubleshooting Guide** 

If you encounter issues at each step, check: 

Part 1: - Is SQLite3 working on your system? - Does the database file get created? - Can you query the tables directly? 

Part 2: - Is your OPENAI_API_KEY correctly set? - Are you getting valid SQL from OpenAI? - Check the schema format 

Part 3: - Are your SQL queries valid? - Is the database connection working? - Are you handling all error cases? 

Part 4: - Are results properly formatted? - Is the validation working? - Are complex queries handled correctly? 

**Extension Activities (if time permits)** 

Add result formatting improvements: 

- Pretty table output 
- CSV export option 
- JSON format option 
- Add query analysis: 
- Execution time measurement 
- Query plan explanation 
- Performance optimization suggestions 
- Enhance safety features: 
- Query complexity limits 
- Resource usage checks 
- Input sanitization 

Remember: Never expose your API keys or database credentials in your code! 

**Testing Requirements** 

1. Basic Queries 
- Simple SELECT statements 
- Aggregations 
- JOINs 
- Edge Cases 
- Invalid questions 
- Complex queries 
- Error handling 
- Results 
- Proper formatting 
- Error messages 
- Query validation 

**Alternative Database Setup** 

For those wanting to use other databases: 

MySQL 

```shell
docker run --name mysql -e MYSQL_ROOT_PASSWORD=password -p 3306:3306 -d mysql:8 
```

PostgreSQL 

```shell
docker run --name postgres -e POSTGRES_PASSWORD=password -p 5432:5432 -d postgres:13 
```

Cloud SQL 
- AWS RDS setup instructions 
- Google Cloud SQL setup 
- Azure Database setup 

**Submission Requirements** 

Create a document containing: 

- Working code 
- Example queries and results 
- Any improvements implemented 
- Brief explanation of error handling 

**Remember:** 

- Keep API keys secure 
- Don’t expose database credentials 
- Test thoroughly before submitting 

**Please note** 

Note: The field of AI is evolving rapidly. By the time you work on these exercises: 

- APIs and services mentioned may have changed or been discontinued 
- Free tiers might no longer be available 
- Pricing structures could be different 
- Python libraries may have new versions with different syntax 
- Example code might need adaptation 

The core concepts remain valid, but you may need to: 

- Find alternative services 
- Adapt code to current API versions 
- Use different model versions 
- Modify prompts for newer models 
- Research current best practices 

This is normal in AI development. Learning to adapt to these changes is part of working with AI tools. 

**Resources**: 

- Check current documentation 
- Review service status pages 
- Join developer communities 
- Research alternatives 


**Estimated time:** 60-90 minutes

**Submission Instructions:**

- Select the **Start Assignment** button at the top right of this page.
- Upload your answers in the form of a Word or PDF file.
- Select the **Submit Assignment** button to submit your responses.

*This is a graded and counts towards programme completion. You may attempt this assignment only once.*


## Import Libraries and Load Environment


In [1]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import sqlite3

# LangChain imports
from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent, initialize_agent, AgentType
from langchain.tools import tool # for @tool
# from langchain_core.tools import Tool # Traditional Tool class
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate

# Transformers for Qwen
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# OpenAI Models: from langchain_openai import ChatOpenAI
# Gemini Models: from langchain_google_palm import ChatGooglePalm
# Ollama Models: from langchain_ollama import ChatOllama


C:\Users\Alkaff\anaconda3\envs\lc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

## 📌 Part 1: Setting Up the Database

In [3]:
# !!! Setup Database - RUN ONLY ONCE !!!
# --------------------------------------
 
def setup_database(): 
    conn = sqlite3.connect('company.db') 
    c = conn.cursor() 
     
    # Create sample tables 
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS employees ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            department TEXT, 
            salary REAL 
        ) 
    ''') 
     
    c.execute(''' 
        CREATE TABLE IF NOT EXISTS departments ( 
            id INTEGER PRIMARY KEY, 
            name TEXT, 
            budget REAL 
        ) 
    ''') 
     
    # Insert sample data 
    c.execute("INSERT OR IGNORE INTO employees VALUES (1, 'John Doe', 'Engineering', 75000)") 
    c.execute("INSERT OR IGNORE INTO employees VALUES (2, 'Jane Smith', 'Marketing', 65000)") 
    c.execute("INSERT OR IGNORE INTO departments VALUES (1, 'Engineering', 1000000)") 
    c.execute("INSERT OR IGNORE INTO departments VALUES (2, 'Marketing', 500000)") 
     
    conn.commit() 
    conn.close() 


In [4]:
# !!! Setup Database - RUN ONLY ONCE !!!
# --------------------------------------

setup_database()

In [5]:
# Testing DB
# ----------

# Verify the setup 
conn = sqlite3.connect('company.db') 
c = conn.cursor() 
 
print("Employees table:") 
c.execute("SELECT * FROM employees") 
print(c.fetchall()) 
 
print("\nDepartments table:") 
c.execute("SELECT * FROM departments") 
print(c.fetchall()) 
 
conn.close()


Employees table:
[(1, 'John Doe', 'Engineering', 75000.0), (2, 'Jane Smith', 'Marketing', 65000.0)]

Departments table:
[(1, 'Engineering', 1000000.0), (2, 'Marketing', 500000.0)]


## 📌 Part 2: Creating the SQL Generator

- **Note:** I have decided to use an open source model Qwen 1.5 1.8b chat instead of OpenAI


### Step 1: Load the Qwen 1.5 1.8b Chat model

In [6]:
# Load LLM - Qwen 1.5 1.8B Chat
# -----------------------------

# Load Model
model_id = "Qwen/Qwen1.5-1.8B-Chat"
tok  = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            device_map="auto",          # "cuda" if GPU, else "cpu"
            torch_dtype="auto"
        )

# Create Qwen Pipeline
qwen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    #max_new_tokens=512,
    temperature=0.2,
    #device=0  # device=0 for GPU
)

# Create the LLM
llm = HuggingFacePipeline(pipeline=qwen_pipeline)


Device set to use cpu
C:\Users\Alkaff\AppData\Local\Temp\ipykernel_14168\3132568482.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qwen_pipeline)


### Step 2: Create the Agent by using the JSON-based approach


In [7]:
# SQL Agent
# ---------

schema_text = """
Table: employees
Columns:
- id (INTEGER PRIMARY KEY)
- name (TEXT)
- department (TEXT)
- salary (REAL)

Table: departments
Columns:
- id (INTEGER PRIMARY KEY)
- name (TEXT)
- budget (REAL)
"""

prompt_sql_text = """
You are a SQL expert assistant.
Given the following database schema:

{schema}

Generate a valid SQL query that answers this question:

"{question}"

IMPORTANT: Return ONLY the SQL query prefixed by "Answer:" and no other additional text. At the end of the query, the semicolon must exist
"""

prompt_sql = PromptTemplate.from_template(prompt_sql_text)


In [10]:
# Testing Sample
# --------------

prompt1 = "List all employees"
prompt_obj = prompt_sql.format(
    schema=schema_text, 
    question=prompt1
)

# Clean up the result
res1 = llm(prompt_obj)
res1 = res1.split("Answer:")[-1].strip()
res1 = res1.split(";")[0].strip()
res1 = res1.replace("```sql", "").replace("```SQL", "").replace("```", "").replace(";", "").strip()

print(f"prompt={prompt1}")
print(res1)

print()

prompt2 = "Show departments with budgets over 750000"
prompt_obj = prompt_sql.format(
    schema=schema_text, 
    question=prompt2
)

# Clean up the result
res2 = llm(prompt_obj)
res2 = res2.split("Answer:")[-1].strip()
res2 = res2.split(";")[0].strip()
res2 = res2.replace("```sql", "").replace("```SQL", "").replace("```", "").replace(";", "").strip()

res2 = llm(prompt_obj)
res2 = res2.split("Answer:")[-1].strip()
print(f"prompt={prompt2}")
print(res2)


C:\Users\Alkaff\AppData\Local\Temp\ipykernel_14168\1260074064.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res1 = llm(prompt_obj)


prompt=List all employees
SELECT employees.name FROM employees

prompt=Show departments with budgets over 750000
SELECT departments.name FROM departments WHERE departments.budget > 750000;


## 📌 Part 3: Implementing Query Execution

In [11]:
# Helper Functions to Execute Query
# ---------------------------------

# Validation - Ensure NO Harmful Queries are Generated
def validate_sql(sql): 
    # Basic safety checks 
    sql_lower = sql.lower() 
    if any(word in sql_lower for word in ['drop', 'delete', 'update', 'insert']): 
        raise ValueError("Only SELECT queries are allowed") 
    return sql 

# Executes the Query
def execute_query(sql): 
    sql = validate_sql(sql) 
    conn = sqlite3.connect('company.db') 
    try: 
        cursor = conn.cursor() 
        cursor.execute(sql) 
        results = cursor.fetchall() 
        return results 
    except Exception as e: 
        return f"Error: {str(e)}" 
    finally: 
        conn.close() 

# Converts the Resultset into Readable Text
def format_results(results): 
    if not isinstance(results, list): 
        return str(results) 
    if not results: 
        return "No results found" 
     
    # If it's a single column result 
    if len(results[0]) == 1: 
        return "\n".join([str(row[0]) for row in results]) 
     
    # For multiple columns, try to format as a table 
    # Get column names from the first result 
    if isinstance(results[0], tuple): 
        # Format each row with proper spacing 
        formatted_rows = [] 
        for row in results: 
            row_items = [] 
            for item in row: 
                if isinstance(item, float): 
                    row_items.append(f"${item:,.2f}" if "salary" in str(row) or "budget" in str(row) else f"{item:.2f}") 
                else: 
                    row_items.append(str(item)) 
            formatted_rows.append("\t".join(row_items)) 
        return "\n".join(formatted_rows) 
     
    return "\n".join([str(row) for row in results])

In [12]:
# Assembled SQL Function
# ----------------------

def sql_exec(query):
    res = ""
    try:
        validate_sql(query)
        rs = execute_query(query)
        res = format_results(rs)
    except ValueError as e:
        res = "Only SELECT queries are allowed"
    except Exception as e:
        res = str(e)
    return res


In [13]:
# Testing Sample
# --------------

query1 = "SELECT * FROM employees WHERE salary > 70000"
res1 = sql_exec(query1)
print(f"query1={query1}")
print(f"res1=\n{res1}")

query2 = "SELECT employees.name FROM employees"
res2 = sql_exec(query2)
print(f"query2={query2}")
print(f"res2=\n{res2}")


query1=SELECT * FROM employees WHERE salary > 70000
res1=
1	John Doe	Engineering	75000.00
query2=SELECT employees.name FROM employees
res2=
John Doe
Jane Smith


In [14]:
# Testing Sample 
# --------------

query3 = "DROP TABLE employees"
res3 = sql_exec(query3)
print(f"query3={query3}")
print(f"res3 formatted=\n{res3}")

query3=DROP TABLE employees
res3 formatted=
Only SELECT queries are allowed


## 📌 Part 4: Creating the Complete Agent

In [15]:
# SQL Agent
# ---------

def sql_agent(user_query):
    try:
        # Prepare the schema text
        schema_text = """
        Table: employees
        Columns:
        - id (INTEGER PRIMARY KEY)
        - name (TEXT)
        - department (TEXT)
        - salary (REAL)

        Table: departments
        Columns:
        - id (INTEGER PRIMARY KEY)
        - name (TEXT)
        - budget (REAL)
        """

        # Fill the prompt template
        prompt_obj = prompt_sql.format(
            schema=schema_text,
            question=user_query
        )

        # Get SQL from the LLM
        sql_raw = llm(prompt_obj)
        # print(f"\n===\n{sql_raw}\n===\n") # For debugging

        # Clean the SQL string
        sql_cleaned = sql_raw.split("Answer:")[-1]
        sql_cleaned = sql_cleaned.split(";")[0].strip()
        sql_cleaned = sql_cleaned.replace("```sql", "").replace("```SQL", "").replace("```", "").replace(";", "").strip()

        print(f"Generated SQL:\n{sql_cleaned}\n")

        # Execute and format the result
        result = sql_exec(sql_cleaned)

        return result

    except Exception as e:
        return f"Error in sql_agent: {e}"


In [18]:
# Test Queries
# ------------

test_questions = [ 
    "From the employees table, what is the average salary of the employees in each department?", # "What is the average salary of all employees in each department?", 
    "List the top 1 department with the highest budget?", # "Which department has the highest budget?", 
    "List all employee names earning more than 70000", 
    "DROP TABLE employees"  # This should be caught by validation 
] 
 
for question in test_questions: 
    print(f"\nQuestion: {question}") 
    ans = sql_agent(question)
    print(f"Answer:\n{ans}") 
    print("="*50)




Question: From the employees table, what is the average salary of the employees in each department?
Generated SQL:
SELECT 
    department,
    AVG(salary) AS avg_salary
FROM 
    employees
GROUP BY 
    department

Answer:
Engineering	75000.00
Marketing	65000.00

Question: List the top 1 department with the highest budget?
Generated SQL:
SELECT 
    d.name AS department_name,
    MAX(d.budget) AS max_budget
FROM 
    departments d
GROUP BY 
    d.id
ORDER BY 
    max_budget DESC
LIMIT 1

Answer:
Engineering	1000000.00

Question: List all employee names earning more than 70000
Generated SQL:
SELECT name FROM employees WHERE salary > 70000 GROUP BY name ORDER BY SUM(salary) DESC

Answer:
John Doe

Question: DROP TABLE employees
Generated SQL:
Drop table employees

Answer:
Only SELECT queries are allowed


## 🏁 Conclusion

For this assignment, I decided to **challenge myself** by using an **open-source model (Qwen 1.5 Chat)** instead of the recommended OpenAI GPT-4. Key observations and adaptations:

- ✅ **Model Choice:** Used Qwen 1.5 Chat to test open-source compatibility with SQL generation tasks.
- ⚠️ **Challenge:** The model frequently produced verbose outputs, explanations, or formatting that conflicted with expected clean SQL strings.
- ✅ **Prompt Engineering:** Updated the prompt to:
  - Force the answer to start with `Answer:`.
  - End the SQL query with a semicolon.
  - Omit any explanations or markdown.
- ✅ **Post-Processing:** Added logic to:
  - Split and clean the output reliably by extracting only the text after `Answer:`.
  - Trim any extra semicolons or whitespace.
- ✅ **Question Design:** Rephrased some natural language questions to be simpler and more explicit to help Qwen generate correct SQL.
- ✅ **Validation:** Implemented safeguards to:
  - Allow only `SELECT` queries.
  - Reject any destructive statements (`DROP`, `DELETE`, etc.).
- ✅ **Result:** This approach allowed:
  - Consistent, executable SQL queries.
  - Safe and clean formatting of results.
  - A reliable pipeline despite the model’s limitations.
- ✅ **Takeaway:** Careful prompt design and strict output parsing are essential when using smaller open-source LLMs in structured tasks like SQL generation.
